In [1]:
import numpy as np
from pathlib import Path
import SimpleITK as sitk
import pandas as pd
from glob import glob
from tqdm import tqdm

In [6]:
csv='/home/kakeya/Desktop/higuchi/20191107/output/csv/deeds_statistics.csv'
df=pd.read_csv(csv,index_col=0)

In [51]:
df['slices'].max()

127

In [8]:
df.describe()

,level_0,index,count,count_CCR,count_cys,lumi_mean,right,left,slices,st,en
count,114.000000,114.000000,1.140000e+02,1.140000e+02,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,56.500000,72.587719,4.763634e+05,8.772718e+04,13561.605263,118.406126,0.622807,0.543860,76.245614,0.035088,65.403509
std,33.052988,46.719235,2.240970e+05,1.850868e+05,42481.319823,32.423571,0.486824,0.500272,16.321201,0.184814,15.051957
min,0.000000,1.000000,1.695010e+05,0.000000e+00,0.000000,27.597880,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.250000,31.250000,3.481402e+05,5.730500e+03,0.000000,97.212787,0.000000,0.000000,65.000000,0.000000,55.000000
50%,56.500000,68.500000,4.257560e+05,1.731850e+04,186.500000,116.937299,1.000000,1.000000,73.000000,0.000000,63.000000
75%,84.750000,112.750000,5.369995e+05,9.374425e+04,4129.500000,141.700137,1.000000,1.000000,85.000000,0.000000,71.000000
max,113.000000,159.000000,1.855627e+06,1.410687e+06,331534.000000,188.365812,1.000000,1.000000,127.000000,1.000000,117.000000


In [17]:
df['lesion']=df['count_CCR']+df['count_cys']
dst_df=df.sort_values(['lesion']).head(10)

In [29]:
src_df=df.query('0<count_CCR<10000').query('0<count_cys<10000').sort_values('lesion').head(20)

In [35]:
dst_df

,level_0,index,count,count_CCR,count_cys,lumi_mean,right,left,slices,st,en,lesion
32,32,35,529584,0.0,171.0,106.932345,0,0,100,0,67,171.0
33,33,36,342846,0.0,318.0,145.806030,0,0,100,0,70,318.0
74,74,96,180731,517.0,0.0,133.310096,0,1,64,0,54,517.0
96,96,127,341765,637.0,0.0,114.909110,0,1,76,0,70,637.0
93,93,124,257499,699.0,0.0,158.450363,1,0,63,0,53,699.0
6,6,7,529050,0.0,928.0,168.719919,0,0,100,0,59,928.0
2,2,3,297822,1092.0,0.0,150.493379,1,0,65,0,55,1092.0
19,19,21,315826,1831.0,0.0,140.291876,0,1,60,0,49,1831.0
22,22,24,389116,0.0,1843.0,129.923105,0,0,100,0,58,1843.0
30,30,33,346307,2495.0,0.0,101.828456,0,1,65,0,55,2495.0


In [46]:
#dst:clear_kidney,src_df:
for src_idx,src in src_df.iterrows():
    ratio=np.inf
    for dst_idx,dst in dst_df.iterrows():
        if src['count']>dst['count']:
            cratio=src['count']/dst['count']
        else:
            cratio=dst['count']/src['count']
        
        if src['lumi_mean']>dst['lumi_mean']:
            lratio=src['lumi_mean']/dst['lumi_mean']
        else:
            lratio=dst['lumi_mean']/src['lumi_mean']            
        tmp_ratio=lratio+cratio
        if ratio>tmp_ratio:

            ratio=tmp_ratio
            src_df.at[src_idx,'dst_mean']=dst_df.at[dst_idx,'lumi_mean']
            src_df.at[src_idx,'dst_count']=dst_df.at[dst_idx,'count']
            src_df.at[src_idx,'dst_slices']=dst_df.at[dst_idx,'slices']
            
    
            src_df.at[src_idx,'ratio']=ratio
            src_df.at[src_idx,'dst_idx']=dst_idx
            
src_df.head()

,level_0,index,count,count_CCR,count_cys,lumi_mean,right,left,slices,st,en,lesion,ratio,dst_idx,dst_mean,dst_count,dst_slices
4,4,5,395548,1847.0,955.0,112.893234,1,1,85,0,75,2802.0,2.167379,22.0,129.923105,389116.0,100.0
111,111,154,245366,1762.0,1077.0,147.857389,1,0,60,1,48,2839.0,2.121092,93.0,158.450363,257499.0,63.0
57,57,69,382154,4125.0,35.0,179.086175,1,0,70,0,60,4160.0,2.342902,33.0,145.806030,342846.0,100.0
44,44,53,363764,4088.0,140.0,156.321568,1,1,69,0,68,4228.0,2.133133,33.0,145.806030,342846.0,100.0
34,34,37,365767,4566.0,16.0,101.087569,1,0,74,0,64,4582.0,2.063522,30.0,101.828456,346307.0,65.0


In [62]:
path='./dees_pair.txt'
with open(path,mode='a') as f:
    for src,dst in zip(src_df['index'],src_df['dst_idx']):
        #print(srcdst)
        print(f'{str(src).zfill(5)} {str(int(dst)).zfill(5)}')
    

00005 00022
00154 00093
00069 00033
00053 00033
00037 00030
00062 00019
00095 00022
00019 00019
00103 00022
00068 00006
00097 00033
00122 00032
00018 00096
00015 00096
00063 00030
00009 00030
00149 00022
00055 00006
00135 00033
00002 00033


In [44]:
src_df.to_csv('../output/csv/dees_pairs.csv')

,level_0,index,count,count_CCR,count_cys,lumi_mean,right,left,slices,st,en,lesion,ratio,dst_idx,dst_mean,dst_count
4,4,5,395548,1847.0,955.0,112.893234,1,1,85,0,75,2802.0,2.167379,22.0,129.923105,389116.0
111,111,154,245366,1762.0,1077.0,147.857389,1,0,60,1,48,2839.0,2.121092,93.0,158.450363,257499.0
57,57,69,382154,4125.0,35.0,179.086175,1,0,70,0,60,4160.0,2.342902,33.0,145.806030,342846.0
44,44,53,363764,4088.0,140.0,156.321568,1,1,69,0,68,4228.0,2.133133,33.0,145.806030,342846.0
34,34,37,365767,4566.0,16.0,101.087569,1,0,74,0,64,4582.0,2.063522,30.0,101.828456,346307.0
50,50,62,279766,4613.0,200.0,134.649975,0,1,62,0,52,4813.0,2.170794,19.0,140.291876,315826.0
73,73,95,413974,5118.0,50.0,148.917519,0,1,114,0,104,5168.0,2.210081,22.0,129.923105,389116.0
18,18,19,270279,5632.0,36.0,134.975581,1,0,63,0,53,5668.0,2.207906,19.0,140.291876,315826.0
79,79,103,431205,6215.0,540.0,148.136225,1,1,70,0,60,6755.0,2.248350,22.0,129.923105,389116.0
56,56,68,438974,1964.0,5105.0,157.945218,1,0,77,0,67,7069.0,2.273415,6.0,168.719919,529050.0


In [65]:
paths=glob('/home/kakeya/ssd/deeds_data_130/*/SE2.nii.gz')
for path in paths:
    im=sitk.ReadImage(path)
    arr=sitk.GetArrayFromImage(im)
    print(path)
    print(arr.shape)

/home/kakeya/ssd/deeds_data_130/00023/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00008/SE2.nii.gz
(73, 512, 512)
/home/kakeya/ssd/deeds_data_130/00045/SE2.nii.gz
(87, 512, 512)
/home/kakeya/ssd/deeds_data_130/00101/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00041/SE2.nii.gz
(66, 512, 512)
/home/kakeya/ssd/deeds_data_130/00002/SE2.nii.gz
(67, 512, 512)
/home/kakeya/ssd/deeds_data_130/00104/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00134/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00007/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00074/SE2.nii.gz
(58, 512, 512)
/home/kakeya/ssd/deeds_data_130/00072/SE2.nii.gz
(69, 512, 512)
/home/kakeya/ssd/deeds_data_130/00036/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00150/SE2.nii.gz
(130, 512, 512)
/home/kakeya/ssd/deeds_data_130/00095/SE2.nii.gz
(114, 512, 512)
/home/kakeya/ssd/deeds_data_130/00038/SE2.nii.gz
(66, 512, 512)
/home/kakeya/ssd/deeds_data_130/

SystemError: <built-in function ReadImage> returned a result with an error set